In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install tensorflow-docs

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.6/183.6 kB 5.5 MB/s eta 0:00:00


In [3]:
from tensorflow_docs.vis import embed
from tensorflow import keras
from imutils import paths

import matplotlib.pyplot as plt
import imageio
 

import numpy as np
import cv2
import os
import pandas as pd
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score

In [4]:
IMG_SIZE = 224
BATCH_SIZE = 64
EPOCHS = 10

MAX_SEQ_LENGTH = 20
NUM_FEATURES = 2048

In [5]:
df = pd.read_csv("/content/drive/MyDrive/PFA/videos/video_train.csv")
val_df = pd.read_csv("/content/drive/MyDrive/PFA/videos/video_val.csv")
test_df = pd.read_csv("/content/drive/MyDrive/PFA/videos/video_test.csv")

print(f"Total videos for validating: {len(val_df)}")
print(f"Total videos for testing: {len(test_df)}")



Total videos for validating: 32
Total videos for testing: 33


In [6]:
# Define paths to video files and corresponding labels
video_paths = df["video_path"].tolist()
labels = df["F1"].tolist()

# Define parameters
image_size = (224, 224)  # Size of input frames
num_classes = len(set(labels))  # Number of distinct classes in your dataset

# Extract video frames and preprocess
def extract_frames(video_path):
    frames = []
    cap = cv2.VideoCapture(video_path)
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, image_size)
        frame = frame / 255.0  # Normalize pixel values between 0 and 1
        frames.append(frame)
    cap.release()
    return frames

# Extract frames for all videos
all_frames = [extract_frames(path) for path in video_paths]

# Split data into training and testing sets
train_frames, test_frames, train_labels, test_labels = train_test_split(
    all_frames, labels, test_size=0.2, random_state=42)

# Convert frames to numpy arrays
train_frames = np.array(train_frames)
test_frames = np.array(test_frames)

# Load pretrained CNN model
base_model = MobileNetV2(include_top=False, weights='imagenet', input_shape=(224, 224, 3))

# Add classification head
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze base model layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(train_frames, train_labels, epochs=10, batch_size=16)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(test_frames, test_labels)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

# Evaluate the model
test_predictions = np.argmax(model.predict(test_frames), axis=1)
f1 = f1_score(test_labels, test_predictions, average='weighted')
print("F1 Score:", f1)

# Make predictions on new videos
new_video_paths = ["path/to/new/video1.mp4", "path/to/new/video2.mp4"]  # Paths to new videos
new_frames = [extract_frames(path) for path in new_video_paths]
new_frames = np.array(new_frames)
predictions = model.predict(new_frames)
predicted_labels = np.argmax(predictions, axis=1)

# Print predicted labels
for path, label in zip(new_video_paths, predicted_labels):
    print(f"Video: {path} - Predicted Label: {label}")

# Generate classification report
classification_rep = classification_report(test_labels, test_predictions)
print(classification_rep)

<ipython-input-6-c56feddb2bd1>:31: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  train_frames = np.array(train_frames)
<ipython-input-6-c56feddb2bd1>:32: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  test_frames = np.array(test_frames)


9406464/9406464 [==============================] - 0s 0us/step


ValueError: ignored